In [5]:
from typing import List
import csv
import json
import argparse
import os
import pandas as pd
class Turn():
    def __init__(self, is_agent : bool, sentences : List[str]):
        self._is_agent = is_agent
        self.sentences = sentences

    def is_agent(self):
        return self._is_agent

    def get_sentences(self):
        return self.sentences

    def get_json(self) -> json:
        result = {
            'is_agent' : self.is_agent(),
            'sentences' : self.sentences
        }
        return json.dumps(result)

    def __str__(self):
        return ('Agent:\t' if self.is_agent() else 'Customer:\t')  + ' '.join(self.sentences)

class Dialog():
    def __init__(self, dialog_id: str, turns: List[Turn]):
        self.dialog_id = dialog_id
        self.turns = turns

    def get_dialog_id(self) -> str:
        return self.dialog_id

    def get_turns(self) -> List[Turn]:
        return self.turns

    def get_json(self) -> json:
        turn_list = [json.loads(turn.get_json()) for turn in self.turns]
        result = {
            'dialog_id' : self.dialog_id,
            'turns' : turn_list
        }
        return json.dumps(result)

    def __str__(self):
        result = self.dialog_id + '\n'
        for turn in self.turns :
            result += '\t' + str(turn) + '\n'
        return result


class DialogWithSummaries():
    def __init__(self, dialog_id: str, turns: List[Turn],
                 extractive_summaries: List[List[Turn]],
                 abstractive_summaries: List[List[str]]):
        self.dialog = Dialog(dialog_id, turns)
        self.extractive_summaries = extractive_summaries
        self.abstractive_summaries = abstractive_summaries

    def get_dialog(self) -> Dialog:
        return self.dialog

    def get_extractive_summaries(self) -> List[List[Turn]]:
        return self.extractive_summaries

    def get_abstractive_summaries(self) -> List[List[str]]:
        return self.abstractive_summaries

    def get_json(self) -> json:
        dialog = json.loads(self.dialog.get_json())
        extractive_summaries_to_json = list()
        for summ in self.extractive_summaries:
            summ_json = [json.loads(turn.get_json()) for turn in summ]
            extractive_summaries_to_json.append(summ_json)
        abstractive_summaries_to_json = list()
        for summ in self.abstractive_summaries:
            abst_json = [txt for txt in summ ]
            abstractive_summaries_to_json.append(abst_json)

        result = {
            'dialog' : dialog,
            'summaries' : {
                'extractive_summaries' : extractive_summaries_to_json,
                'abstractive_summaries' : abstractive_summaries_to_json

            }
        }

        return json.dumps(result)

    def __str__(self):
        result = str(self.dialog)
        result += '\n'
        # Extractive summaries:
        result += 'Extractive summaries:\n'
        result += '=-=-=-=-=-=-=-=-=-=-=\n'
        for cnt,extractive_summary in enumerate(self.extractive_summaries):
            result += ('{0}:\n'.format(cnt))
            for turn in extractive_summary:
                result += '\t' + str(turn) +'\n'
            result += '\n'

        # Abstractive summaries:
        result += 'Abstractive summaries:\n'
        result += '=-=-=-=-=-=-=-=-=-=-=\n'
        for cnt,abstractive_summary in enumerate(self.abstractive_summaries):
            result += ('{0}:\n'.format(cnt))
            if (abstractive_summary is not None):
              result += '\t' + ' '.join(abstractive_summary) +'\n'
            result += '\n'
        return result


class TweetSumProcessor():
    def __init__(self, path_to_twitter_kaggle_file):
        self.tweet_id_to_content = dict()
        with open(path_to_twitter_kaggle_file) as f:
            csv_reader = csv.reader(f)
            # We skip header
            next(csv_reader)
            for line in csv_reader:                         
              tweet_id = str(line[0])                
              in_bound = line[2]                
              text = line[4]
              self.tweet_id_to_content[tweet_id] = (in_bound, text)

    def __get_turn(self, tweet_id : str, sentence_offsets : List[dict]) -> Turn:   
        if self.tweet_id_to_content.get(str(tweet_id)) is not None:
          content = self.tweet_id_to_content[str(tweet_id)]
          in_bound = content[0]
          text = content[1]
          sentences = list()
          for offset in sentence_offsets:
              start, end = offset.replace('[', '').replace(']', '').split(',')
              sentence = text[int(start):int(end)]
              sentences.append(sentence)
          turn = Turn(is_agent=('FALSE' == str(in_bound).upper()), sentences=sentences)

          return turn

    def __get_turns(self, tweet_ids_sentence_offsets: List[dict]) -> List[Turn]:
        turns = list()
        for tweet_id_sentence_offset in tweet_ids_sentence_offsets :
            tweet_id = tweet_id_sentence_offset['tweet_id']
            offsets = tweet_id_sentence_offset['sentence_offsets']

            turns.append(self.__get_turn(tweet_id, offsets))

        return turns

    def __get_extractive_summaries(self, annotations) -> List[List[Turn]]:
        extractive_summaries = list()
        for annotation in annotations:
            if 'extractive' in annotation.keys():
                extractive_summary = list()
                extractive_annotation = annotation['extractive']
                if extractive_annotation :
                    for sentence in extractive_annotation :
                        tweet_id = sentence['tweet_id']
                        offset = sentence['sentence_offset']
                        turn = self.__get_turn(tweet_id, [offset])
                        extractive_summary.append(turn)
                    extractive_summaries.append(extractive_summary)

        return extractive_summaries

    @staticmethod
    def __get_abstractive_summaries(annotations) -> List[List[str]]:
        summaries = list()
        for annotation in annotations:
            if 'abstractive' in annotation.keys():
                abstractive_summary = annotation['abstractive']
                summaries.append(abstractive_summary)
        return summaries

    def get_dialog_with_summaries(self, tweet_sum_lines: List[str]) -> List[DialogWithSummaries]:
        result = list()        
        for tweet_sum_line in tweet_sum_lines:
            tweet_sum_dict = json.loads(tweet_sum_line)            
            conversation_id = tweet_sum_dict['conversation_id']
            tweet_ids_sentence_offsets = tweet_sum_dict['tweet_ids_sentence_offset']
            annotations = tweet_sum_dict['annotations']
            turns = self.__get_turns(tweet_ids_sentence_offsets)          
            extractive_summaries = self.__get_extractive_summaries(annotations)
            abstractive_summaries = self.__get_abstractive_summaries(annotations)

            dialog_with_summaries = DialogWithSummaries(dialog_id=conversation_id,
                                                        turns=turns,
                                                        extractive_summaries=extractive_summaries,
                                                        abstractive_summaries=abstractive_summaries)
            result.append(dialog_with_summaries)
        return result

In [11]:
TWEETSUM_FILE_PATH = os.getcwd() + '/final_train_tweetsum.jsonl'
# Download datafrom https://www.kaggle.com/thoughtvector/customer-support-on-twitter
TWCS_FILE_PATH =  os.getcwd() + '/twcs.csv'
processor = TweetSumProcessor(TWCS_FILE_PATH)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1887: character maps to <undefined>

In [12]:
def file_process(TWEETSUM_FILE_PATH):
  conversation_list = []
  extractive_list = []
  abstractive_list = []
  with open(TWEETSUM_FILE_PATH) as f:        
    dialog_with_summaries = processor.get_dialog_with_summaries(f.readlines())    
    for dialog_with_summary in dialog_with_summaries:                            
        tmp = dialog_with_summary
        string_format = str(dialog_with_summary)
        conversation = string_format[string_format.find("\n\t"):string_format.find("Extractive")].replace('\n', '').replace('\t', '')
        extractive = string_format[string_format.find("Extractive"):string_format.find("Abstractive")].replace("Extractive summaries:", '').replace('=-', '').replace('\n', '').replace('\t', '').replace('=0:', '')       
        abstractive = string_format[string_format.find("Abstractive"):].replace('Abstractive summaries', '').replace('=-', '').replace('\n', '').replace('\t', '').replace(':=0:', '')
        conversation_list.append(conversation)
        extractive_list.append(extractive)
        abstractive_list.append(abstractive)
  return conversation_list, extractive_list, abstractive_list

In [13]:
TWEETSUM_FILE_PATH = os.getcwd() + '/final_train_tweetsum.jsonl'
conversation_list, extractive_list, abstractive_list = file_process(TWEETSUM_FILE_PATH)
data = {'conversation':conversation_list, 'extractive summary':extractive_list, 'abstractive summary':abstractive_list}
df_train = pd.DataFrame(data)
pd.set_option('display.max_colwidth', None)
df_train.head(1)

NameError: name 'processor' is not defined

In [14]:
TWEETSUM_FILE_PATH = os.getcwd() + '/final_valid_tweetsum.jsonl'
conversation_list, extractive_list, abstractive_list = file_process(TWEETSUM_FILE_PATH)
data = {'conversation':conversation_list, 'extractive summary':extractive_list, 'abstractive summary':abstractive_list}
df_val = pd.DataFrame(data)
pd.set_option('display.max_colwidth', None)
df_val.head(1)

NameError: name 'processor' is not defined

In [13]:
TWEETSUM_FILE_PATH = os.getcwd() + '/final_test_tweetsum.jsonl'
conversation_list, extractive_list, abstractive_list = file_process(TWEETSUM_FILE_PATH)
data = {'conversation':conversation_list, 'extractive summary':extractive_list, 'abstractive summary':abstractive_list}
df_test = pd.DataFrame(data)
pd.set_option('display.max_colwidth', None)


In [20]:
discard = ['None']
df_test = df_test[~df_test.conversation.str.contains('|'.join(discard))]

In [21]:
df_test

,conversation,extractive summary,abstractive summary
14,"Customer:@hulu_support it seems like since th last update volume on the Apple TV has gone down. So we have to turn it up more. This is on all our TVsAgent:@128399 That's odd behavior! Which generation Apple TV do you have? Are you noticing lower volume on all content or a certain show/movie?Customer:@hulu_support Sorry for not replying. It was mostly CBS and other random ones. But seems to be better and back to normal now.Agent:@128399 Apologies for the delay on our end! We're so glad to hear things seem to be back to normal! Let us know if this happens again.Customer:@hulu_support Well it seems like some CBS shows are quieter then other shows/channels. Not sure what happened again.Agent:@128399 Thanks for following up! If you let us know which shows, and which generation Apple TV you have, we'll take a closer look.Customer:@hulu_support 4th gen Apple TV 2 of them. Kevin can wait was one of them and it seems most CBS shows at time can be quieter.Agent:@128399 Is the audio lower all throughout playback or does it decrease at a certain point? Let us know if: https://t.co/56JXI4vxlf helps.Customer:@hulu_support Throughout. It seems random on what shows and when it happens. Almost like it comes and goes. But always CBS shows.","Customer:@hulu_support it seems like since th last update volume on the Apple TV has gone down.Agent:Are you noticing lower volume on all content or a certain show/movie?Agent:If you let us know which shows, and which generation Apple TV you have, we'll take a closer look.Customer:Kevin can wait was one of them and it seems most CBS shows at time can be quieter.Agent:@128399 Is the audio lower all throughout playback or does it decrease at a certain point?1:Customer:@hulu_support it seems like since th last update volume on the Apple TV has gone down.Customer:@hulu_support Well it seems like some CBS shows are quieter then other shows/channels.Agent:If you let us know which shows, and which generation Apple TV you have, we'll take a closer look.Customer:@hulu_support 4th gen Apple TV 2 of them.Agent:@128399 Is the audio lower all throughout playback or does it decrease at a certain point?2:Customer:@hulu_support it seems like since th last update volume on the Apple TV has gone down.Customer:So we have to turn it up more.Customer:This is on all our TVsAgent:Are you noticing lower volume on all content or a certain show/movie?Customer:It was mostly CBS and other random ones.Customer:But seems to be better and back to normal now.Customer:@hulu_support Well it seems like some CBS shows are quieter then other shows/channels.Agent:If you let us know which shows, and which generation Apple TV you have, we'll take a closer look.Customer:Kevin can wait was one of them and it seems most CBS shows at time can be quieter.Agent:@128399 Is the audio lower all throughout playback or does it decrease at a certain point?",Customer is complaining about the low volume on Apple TV after updating it. Agent wants to know whether the problem is throughout the playback or at certain points.1:The customer says that the volume on his Apple TV has gone down after the last update. The agent asks which generation Apple Tv does the customer owns and whether audio is lower all throughout playback or decrease at certain point.2:Customer is complaining that after the last update the Apple tv volume has been decreased. Agent asks whether the volume decreased in certain point or throughout the playback and provided the link which may help.
20,"Customer:@115858 @AppleSupport music and podcast “skip” around like a CD, then distorts and clears up in few seconds, only happens after iOS 11Agent:@118095 We're here to help. Did this happen after a specific update, or just recently? Thanks!Customer:@AppleSupport I noticed shortly after the first patch to iOS 11 not sure which one, but here in the past week or two it has gotten worse and more frequentCustomer:@AppleSupport It’s only done it wi